In [1]:
import pandas as pd
import os
import datetime
import numpy as np
import glob
import json
import glob
from haversine import haversine
import gc
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Experian_Audience'

In [2]:
df_experiance=pd.read_csv("/home/jian/BigLots/Experian/data_20191016/Experian_20191001114500.txt",
                         dtype=str,sep="|",usecols=['CustomerNumber','IncomeId','IncomeDesc','HomerentId','HomerentDesc'])
print(df_experiance.shape,df_experiance['CustomerNumber'].nunique())
df_experiance.head(2)

(17626573, 5) 17626573


,CustomerNumber,IncomeId,IncomeDesc,HomerentId,HomerentDesc
0,9,2,between $50k and $100k,1,own house
1,28,3,>= $100k,1,own house


In [3]:
df_income=df_experiance[['IncomeId','IncomeDesc']].drop_duplicates()
print(df_income.shape,df_income['IncomeId'].nunique(),df_income['IncomeDesc'].nunique())

df_home=df_experiance[['HomerentId','HomerentDesc']].drop_duplicates()
print(df_home.shape,df_home['HomerentId'].nunique(),df_home['HomerentDesc'].nunique())

(10, 2) 9 3
(6, 2) 6 2


In [4]:
df_income['IncomeDesc'].unique().tolist()

['between $50k and $100k', '>= $100k', '< $50k', nan]

In [5]:
df_home['HomerentDesc'].unique().tolist()

['own house', 'rents', nan]

In [6]:
list_income_desc=['< $50k','between $50k and $100k', '>= $100k']
list_home_desc=['own house', 'rents']

In [7]:
df_experiance=df_experiance[df_experiance['IncomeDesc'].isin(list_income_desc)]
print(df_experiance.shape)
df_experiance=df_experiance[df_experiance['HomerentDesc'].isin(list_home_desc)]
print(df_experiance.shape)

(17625143, 5)
(17623397, 5)


In [8]:
df_exp_mapping_hash=pd.read_csv("/home/jian/BigLots/Experian/data_20191016/Experian 2019 customer hashed.txt",
                           dtype=str,sep="|")
print(df_exp_mapping_hash.shape,df_exp_mapping_hash['Customer_Num'].nunique(),df_exp_mapping_hash['hashed_customer_num'].nunique())
df_exp_mapping_hash.head(2)

(17626573, 2) 17626573 17626573


,Customer_Num,hashed_customer_num
0,812,313c938e0103b56b43632b702e3e63447fab1f90a4fe89...
1,983,fbe10beedf9d29cf53137ba38859ffd1dbe7642cedb7ef...


In [9]:
df_exp_mapping_hash=df_exp_mapping_hash.rename(columns={"Customer_Num":"CustomerNumber","hashed_customer_num":"customer_id_hashed"})
df_exp_mapping_hash.head(2)

,CustomerNumber,customer_id_hashed
0,812,313c938e0103b56b43632b702e3e63447fab1f90a4fe89...
1,983,fbe10beedf9d29cf53137ba38859ffd1dbe7642cedb7ef...


In [10]:
df_experiance=pd.merge(df_experiance,df_exp_mapping_hash,on="CustomerNumber",how="left")
print(df_experiance.shape,df_experiance['CustomerNumber'].nunique())
df_experiance.head(2)

(17623397, 6) 17623397


,CustomerNumber,IncomeId,IncomeDesc,HomerentId,HomerentDesc,customer_id_hashed
0,9,2,between $50k and $100k,1,own house,19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebab...
1,28,3,>= $100k,1,own house,59e19706d51d39f66711c2653cd7eb1291c94d9b55eb14...


In [11]:
df_experiance=df_experiance[['customer_id_hashed','IncomeDesc','HomerentDesc']]

In [12]:
del df_exp_mapping_hash
gc.collect()

0

In [13]:
list_hashed_id_exp=df_experiance['customer_id_hashed'].tolist()
print(len(list_hashed_id_exp))

17623397


# All master_zip_email

In [14]:
df_email_1012=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/BL_all_master_id_email_zip_closet_store_ZipLabel_upto_20191012_JL_2019-10-23",
               dtype=str,usecols=['customer_id_hashed','email_address_hash','cleaned_zip_cd','closest_store'])
df_email_1012.shape

# Since the Experian data is received in Oct, no need to see the extra new sign ups

(30732606, 4)

In [15]:
df_email_1012.head(2)

,customer_id_hashed,email_address_hash,cleaned_zip_cd,closest_store
0,5ef4d1f4d0bc545e81340e8ad58db181e904392172c05d...,4979adf6b28136f0f0e7fd092b7f20712fe93fe8b21516...,00nan,NaN
1,9c95a62baf8dcc9c33b117853d30cdd516e219c34eeb6f...,8da7290d6a194661c5af68946ff9ff95ec310dabcb9569...,00nan,NaN


In [16]:
del df_email_1012['closest_store']
# update based on recent store list received on Dec.1 
df_unique_cleaned_zip=df_email_1012[['cleaned_zip_cd']].drop_duplicates()
df_unique_cleaned_zip=df_unique_cleaned_zip[df_unique_cleaned_zip['cleaned_zip_cd'].apply(lambda x: x.isdigit())]
print(df_unique_cleaned_zip.shape)

(69011, 1)


In [17]:
df_store_locations=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20191201-133313-746.txt",
                                dtype=str,sep="|",usecols=['location_id','latitude_meas','longitude_meas'])
df_store_locations['latitude_meas']=df_store_locations['latitude_meas'].astype(float)
df_store_locations['longitude_meas']=df_store_locations['longitude_meas'].astype(float)
df_store_locations=df_store_locations[df_store_locations['latitude_meas']!=0]
df_store_locations=df_store_locations[df_store_locations['location_id']!="6990"]
df_store_locations=df_store_locations[['location_id','latitude_meas','longitude_meas']]
df_store_locations['store_coor']=df_store_locations[['latitude_meas','longitude_meas']].values.tolist()
dict_store_location=df_store_locations.set_index(['location_id']).to_dict()['store_coor']

In [18]:
def find_the_cloest_store(input_zip):
    try:
        zip_center=zip_centers[input_zip]
    except:
        return np.nan
    min_dist=np.inf
    output_store_closest=np.nan
    for k,v in dict_store_location.items():
        dist=haversine(zip_center,v,unit="mi")
        if dist<min_dist:
            min_dist=dist
            output_store_closest=k
    return output_store_closest
            

In [19]:
df_unique_cleaned_zip['closest_store']=df_unique_cleaned_zip['cleaned_zip_cd'].apply(lambda x: find_the_cloest_store(x))


In [20]:
df_email_1012=pd.merge(df_email_1012,df_unique_cleaned_zip,on="cleaned_zip_cd",how="left")

# Sales 12 months by id

In [22]:
# apply the rule if ever shopped in a SOTF, the id will be labeled as a SOTF id

SOTF_store_list=pd.read_excel("./BL Store List Report from Dom 11.15.19 354PM.xlsx",dtype=str)
SOTF_store_list=SOTF_store_list[SOTF_store_list['SOTF']=="Yes"]
print(SOTF_store_list.shape)

(505, 23)


In [24]:
list_SOTF=SOTF_store_list['Store'].unique().tolist()
list_SOTF=[x for x in list_SOTF if x.isdigit()]
print(len(list_SOTF))

492


In [25]:
most_recent_week_with_data=datetime.date(2019,12,12)

In [26]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root, file)

In [27]:
list_2019_POS=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
list_2019_POS=[x for x in list_2019_POS if "daily" in x.lower()]
list_2019_POS.sort()

list_2019_POS=[x for x in list_2019_POS if x.split("s/MediaStorm_")[1][:10]>"2019-02-09"] # remove the 1 st week in 2019Q1
print(len(list_2019_POS))
print(list_2019_POS[0])
print(list_2019_POS[-1])

44
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-16/MediaStormDailySales20190219-113605-481.txt
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-14/MediaStormDailySales20191217-195625-000.txt


In [28]:
list_2018_data=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
list_2018_data.sort()
list_2018_data=list_2018_data[-(52-len(list_2019_POS)):]

In [29]:
list_all_POS_52_weeks=list_2018_data+list_2019_POS
print(len(list_all_POS_52_weeks))

52


In [31]:
i_counter=0
df_legacy_ids=df_experiance[['customer_id_hashed']]
df_SOTF_ids=pd.DataFrame()
for file in list_all_POS_52_weeks:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','customer_id_hashed']).drop_duplicates()
    df=df[df['location_id'].isin(list_SOTF)]
    set_SOTF_ids=set(df['customer_id_hashed'].tolist())
    
    df=df_legacy_ids[df_legacy_ids['customer_id_hashed'].isin(set_SOTF_ids)][['customer_id_hashed']]
    df_SOTF_ids=df_SOTF_ids.append(df)
    df_legacy_ids=df_legacy_ids[~df_legacy_ids['customer_id_hashed'].isin(set_SOTF_ids)]
    i_counter+=1
    print(i_counter,datetime.datetime.now())

1 2019-12-18 14:03:26.483527
2 2019-12-18 14:04:18.554037
3 2019-12-18 14:05:08.943594
4 2019-12-18 14:05:54.603499
5 2019-12-18 14:06:53.224294
6 2019-12-18 14:07:45.020022
7 2019-12-18 14:08:25.130011
8 2019-12-18 14:09:05.002745
9 2019-12-18 14:09:48.549790
10 2019-12-18 14:10:30.256811
11 2019-12-18 14:11:17.506252
12 2019-12-18 14:12:04.198777
13 2019-12-18 14:12:49.939036
14 2019-12-18 14:13:39.434639
15 2019-12-18 14:14:27.085664
16 2019-12-18 14:15:10.868845
17 2019-12-18 14:15:53.088827
18 2019-12-18 14:16:45.446092
19 2019-12-18 14:17:25.696923
20 2019-12-18 14:18:16.749522
21 2019-12-18 14:19:01.918870
22 2019-12-18 14:19:43.662077
23 2019-12-18 14:20:16.778674
24 2019-12-18 14:20:49.125316
25 2019-12-18 14:21:35.279217
26 2019-12-18 14:22:25.628774
27 2019-12-18 14:23:09.228503
28 2019-12-18 14:23:59.813039
29 2019-12-18 14:24:36.490897
30 2019-12-18 14:25:19.059656
31 2019-12-18 14:26:02.089482
32 2019-12-18 14:26:52.495692
33 2019-12-18 14:27:26.626268
34 2019-12-18 14:28

In [32]:
print(df_experiance.shape)
print(df_SOTF_ids.shape)
print(df_legacy_ids.shape)

(17623397, 3)
(5261812, 1)
(12361585, 1)


In [33]:
# The kernel always gets died
'''
i_counter=0
df_all_sales_id_by_store=pd.DataFrame()
for file in list_all_POS_52_weeks:
    try:
        week_end_dt=file.split("s/MediaStorm_")[1][:10].replace("-","")
    except:
        week_end_dt=file.split("MediaStormDailySalesHistory")[1][:8]
        
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    # df=df[df['customer_id_hashed'].isin(list_hashed_id_exp)]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(["customer_id_hashed","location_id"])['transaction_id'].count().to_frame().reset_index()
    df_trans=df_trans.rename(columns={"transaction_id":"trans"})
    df_sales=df.groupby(["customer_id_hashed","location_id"])['item_transaction_amt'].sum().to_frame().reset_index()
    df_sales=df_sales.rename(columns={"item_transaction_amt":"sales"})
    df=pd.merge(df_sales,df_trans,on=["customer_id_hashed","location_id"],how="left")
    df['week_end_dt']=week_end_dt
    i_counter+=1
    df_all_sales_id_by_store=df_all_sales_id_by_store.append(df)
    print(i_counter,week_end_dt,datetime.datetime.now())
'''

'\ni_counter=0\ndf_all_sales_id_by_store=pd.DataFrame()\nfor file in list_all_POS_52_weeks:\n    try:\n        week_end_dt=file.split("s/MediaStorm_")[1][:10].replace("-","")\n    except:\n        week_end_dt=file.split("MediaStormDailySalesHistory")[1][:8]\n        \n    df=pd.read_table(file,dtype=str,sep="|",usecols=[\'location_id\',\'transaction_dt\',\'transaction_id\',\'customer_id_hashed\',\'item_transaction_amt\'])\n    # df=df[df[\'customer_id_hashed\'].isin(list_hashed_id_exp)]\n    df[\'item_transaction_amt\']=df[\'item_transaction_amt\'].astype(float)\n    df_trans=df[[\'location_id\',\'transaction_dt\',\'transaction_id\',\'customer_id_hashed\']].drop_duplicates()\n    df_trans=df_trans.groupby(["customer_id_hashed","location_id"])[\'transaction_id\'].count().to_frame().reset_index()\n    df_trans=df_trans.rename(columns={"transaction_id":"trans"})\n    df_sales=df.groupby(["customer_id_hashed","location_id"])[\'item_transaction_amt\'].sum().to_frame().reset_index()\n    df_

In [36]:
df_SOTF_ids['Store_Type']="SOTF"
df_legacy_ids['Store_Type']="Legacy"

df_store_for_id=df_SOTF_ids.append(df_legacy_ids)

In [37]:
del df_SOTF_ids
del df_legacy_ids
gc.collect()

4898

# Final Output

In [44]:
print(df_experiance.shape,df_store_for_id.shape)
df_experiance=pd.merge(df_experiance,df_store_for_id,on="customer_id_hashed",how="left")
print(df_experiance.shape)

(17623397, 3) (17623397, 2)
(17623397, 4)


In [45]:
# Ignored the closest store
df_email_1012.head(2)

,customer_id_hashed,email_address_hash,cleaned_zip_cd,closest_store
0,5ef4d1f4d0bc545e81340e8ad58db181e904392172c05d...,4979adf6b28136f0f0e7fd092b7f20712fe93fe8b21516...,00nan,NaN
1,9c95a62baf8dcc9c33b117853d30cdd516e219c34eeb6f...,8da7290d6a194661c5af68946ff9ff95ec310dabcb9569...,00nan,NaN


In [46]:
df_experiance.head(2)

,customer_id_hashed,IncomeDesc,HomerentDesc,Store_Type
0,19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebab...,between $50k and $100k,own house,SOTF
1,59e19706d51d39f66711c2653cd7eb1291c94d9b55eb14...,>= $100k,own house,Legacy


In [47]:
df_email_1012=df_email_1012[['customer_id_hashed','email_address_hash']]
print(df_email_1012.shape,df_email_1012['customer_id_hashed'].nunique())

(30732606, 2) 30732606


In [48]:
print(df_experiance.shape)
df_experiance=pd.merge(df_experiance,df_email_1012,on="customer_id_hashed",how="left")
print(df_experiance.shape)

(17623397, 4)
(17623397, 5)


In [49]:
df_experiance.head(2)

,customer_id_hashed,IncomeDesc,HomerentDesc,Store_Type,email_address_hash
0,19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebab...,between $50k and $100k,own house,SOTF,dd53075d06b1c045bb7b5756adf7e39d1ef26b24022b17...
1,59e19706d51d39f66711c2653cd7eb1291c94d9b55eb14...,>= $100k,own house,Legacy,bbe01b57dc8fb9c8c9f6722f15836c73ba8eb481279af0...


In [51]:
df_experiance[pd.isnull(df_experiance['email_address_hash'])].shape

(1602143, 5)

In [52]:
1602143/17623397

0.09090999879308172

In [50]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Experian_Audience'

In [53]:
df_experiance.to_csv("./all_experian_id_detail_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [55]:
dict_rename_HHI={">= $100k":"HHI100KHigher","< $50k":"HHI50KLower","between $50k and $100k":"HHI50to100K"}
dict_rename_house={"own house":"HouseOwner","rents":"HouseRents"}

In [59]:
writer_folder_uploaded='/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Experian_Audience/uploaded/'
try:
    os.stat(writer_folder_uploaded)
except:
    os.mkdir(writer_folder_uploaded)

In [62]:
df_summary_count=pd.DataFrame()
i_counter=0
for seg,df_group in df_experiance.groupby(['Store_Type','IncomeDesc','HomerentDesc']):
    
    label_sotf=seg[0]
    label_HHI=dict_rename_HHI[seg[1]]
    label_House=dict_rename_house[seg[2]]
    segment="ExperianSegment_2019Q4_"+"_".join([label_sotf,label_HHI,label_House])
    
    df_group=df_group[['email_address_hash']]
    df_group['segment']=segment
    df_group=df_group[pd.notnull(df_group['email_address_hash'])].drop_duplicates()
    
    i_counter+=1
    count=df_group.shape[0]
    df=pd.DataFrame({"segment":segment,"email_count":count},index=[i_counter])
    df_summary_count=df_summary_count.append(df)
    df_group.to_csv(writer_folder_uploaded+segment+".csv",index=False)

/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [64]:
df_summary_count.to_csv("./summary_experian_audience_2019Q4.csv",index=False)